## HRA-API Documentation

### INDEX
1. [Installation](#install) &#x2705;
2. [Imports](#imports) &#x2705;
3. [Configuration](#config) &#x2705;
4. [Methods](#methods)
    1. [Database Status](#db_status) &#x274C;
    2. [Run a Sparql query](#sparql_get) &#x274C;
    3. [Run a Sparql query (POST)](#sparql_post)
    4. [Get aggregate results / statistics](#agg_results)
    5. [get Spatial Placement Request](#spatial_entity)
    6. [Get all hubmap rui locations](#rui_loc) 
    7. [Get number of ontology term occurrences for a search](#ontology_term)
    8. [Get number of cell type term occurrences for a search](#cell_term_occ)
    9.  [Get ontology term tree nodes](#ontology_nodes)
    10. [Get cell type tree nodes](#cell_nodes)
    11. [Get tissue provider names (for filtering)](#providers)
    12. [Get all reference organs](#ref_organs)
    13. [Get all nodes to form the 3D scene for an organ](#3d_scene)
    14. [Get all nodes to form the 3D scene of reference body, organs, and tissues](#nodes_3d)
    15. [Get technology names (for filtering)](#tech)
    16. [Get Tissue Block Results](#tissue_block)
    17. [Get all GTEx rui locations (if enabled)](#gtex_rui)
    18. [Token](#token_usage)
    

***
[Click here](https://observablehq.com/@cns-iu/ccf-api-documentation-javascript) to view the documentation for ccf-api javascript module.  


<a id='install'></a>
### Installation &#x2705;

#### hra-api-client can be installed via `pip` from [PyPI](https://pypi.org/project/hra-api-client/#description).
***
> `pip install hra-api-client`
***

<a id='imports'></a>
### Imports &#x2705;

We import the hra-api-client as follows:

In [19]:
import hra_api_client

Although we can use hra-api-client for all our tasks, it is easier if we have individual imports as follows:

In [20]:
from hra_api_client.api import v1_api as default_api
from hra_api_client.models.aggregate_count import AggregateCount
from hra_api_client.models.database_status import DatabaseStatus
from hra_api_client.models.error_message import ErrorMessage
from hra_api_client.models.flat_spatial_placement import FlatSpatialPlacement
from hra_api_client.models.get_spatial_placement_request import GetSpatialPlacementRequest
from hra_api_client.models.min_max import MinMax
from hra_api_client.models.ontology_tree import OntologyTree
from hra_api_client.models.sparql_query_request import SparqlQueryRequest
from hra_api_client.models.spatial_entity import SpatialEntity
from hra_api_client.models.spatial_scene_node import SpatialSceneNode
from hra_api_client.models.spatial_search import SpatialSearch
from hra_api_client.models.tissue_block import TissueBlock

For the purposes of this documentation we need some other libraries imported here: 

In [21]:
import time
from pprint import pprint

<a id='config'></a>
### Configuration &#x2705;
ccf-openapi can fetch data from 
> HRA-API: https://apps.humanatlas.io/hra-api/v1 <br> 
> Staging Server: https://apps.humanatlas.io/hra-api--staging/v1 <br>
> You could also use any other instance that you decide to host our code on

You can get more information about our APIs [here](https://apps.humanatlas.io/hra-api/#/).

You'll need to point the host in the configuration to one of the links: 

In [22]:
# may need to remove `v1`
configuration = hra_api_client.Configuration(
    host = "https://apps.humanatlas.io/api" 
)

<a id='api'></a>
### Creating the API Instance &#x2705;
We will need to create an instance of ApiClient from hra-api-client. The configuration we created above will be passed as a parameter for the constructor. <br> <br>
Using the instance of ApiClient we create an instance of DefaultApi from the default_api module we imported above. This instance represents the API interface and provides methods to interact with the API endpoints defined in the OpenAPI specification.

In [23]:
api_client = hra_api_client.ApiClient(configuration)

api_instance = default_api.V1Api(api_client)

<a id='methods'></a>
### Methods

hra-api-client provides multiple methods which we have described below. Each method is described in more detail below : 

***
<a id='db_status'></a>
#### Database Status &#x274C;
On first run the database might take some time to be ready, so we run it on loop and wait until it's ready.
<pre>
Accepted Parameters:
    token - (string) (optional)
</pre>
***
> You can still use the other methods without waiting for the database to be ready, but the initial requests might take a little longer.

In [24]:
# replace dict notation with dot notation
db_ready = False
result = None
while not db_ready:
    result = api_instance.db_status()
    print(result)
    if result.status == 'Ready':
        db_ready = True
    else:
        print('Database not ready yet! Retrying...', result)
        time.sleep(2)
print('Database ready!\n', result)

MaxRetryError: HTTPSConnectionPool(host='apps.humanatlas.io', port=443): Max retries exceeded with url: /api/v1/db-status (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

***
<a id='sparql_get'></a>
#### Run a SPARQL query
<pre>
Accepted Parameters: 
    token - (string) (optional)
    format - (string) (optional) (use only application/json in format, any other formats will result in errors.)
    query - (string) (required)
</pre>

In [25]:
try:
    api_response = api_instance.sparql(query='SELECT * WHERE { ?sub ?pred ?obj . } LIMIT 10', format='application/json')
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

MaxRetryError: HTTPSConnectionPool(host='apps.humanatlas.io', port=443): Max retries exceeded with url: /api/v1/sparql?query=SELECT%20%2A%20WHERE%20%7B%20%3Fsub%20%3Fpred%20%3Fobj%20.%20%7D%20LIMIT%2010&format=application/json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

:warning:
***
<a id='sparql_post'></a>
#### Run a SPARQL query (POST) &#x274C;
<pre>
Accepted Parameters:
    token - (string) (optional)
    format - (string) (optional) (use only application/json in format, any other formats will result in errors.)
    sparql_query_request - { "query": "" } (required)
</pre>

In [26]:
request = {"query": "SELECT * WHERE { ?sub ?pred ?obj . } LIMIT 10"}
# use only application/json in format, any other formats will result in errors.
try:
    # Get aggregate results / statistics
    api_response = api_instance.sparql_post(sparql_query_request=request, format='application/json')
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

MaxRetryError: HTTPSConnectionPool(host='apps.humanatlas.io', port=443): Max retries exceeded with url: /api/v1/sparql?format=application/json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

***
<a id='agg_results'></a>
#### Get aggregate results / statistics
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) 
</pre>

In [27]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.aggregate_results(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

MaxRetryError: HTTPSConnectionPool(host='apps.humanatlas.io', port=443): Max retries exceeded with url: /api/v1/aggregate-results?age=%7B%22min%22%3A%2045.0%2C%20%22max%22%3A%2060.0%7D&bmi=%7B%22min%22%3A%2065.0%2C%20%22max%22%3A%2083.0%7D&cache=true&ontology-terms=http://purl.obolibrary.org/obo/UBERON_0000955&cell-type-terms=http://purl.obolibrary.org/obo/CL_0000000&sex=female&spatial=%7B'x':%2084.0,%20'y':%20146.0,%20'z':%2053.0,%20'radius':%2012.0,%20'target':%20'http://purl.org/ccf/latest/ccf.owl (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

In [ ]:
***
<a id='spatial_entity'></a>
#### Given a SpatialEntity already placed relative to a reference SpatialEntity retrieve a new direct SpatialPlacement to the given SpatialEntity IRI
More Info : https://apps.humanatlas.io/hra-api/#/operations/get-spatial-placement
<pre>
Accepted Parameters:
    get_spatial_placement_request = {
        "target_iri": (string) (required)
        "rui_location": {
            A Spatial Entity defines a bounded cartesian space and the units it is measured in.
            It typically represents a real world thing, e.g., a human body, a human kidney, a tissue section,
            or an individual cell.
            
            "@id": string<uri> (required)
            "@type": (string) (required) (allowed value: SpatialEntity)
            
            "entityId": string<uri> (optional) (Identifier of the entity the spatial object represents)
            "ccf_annotations": array[string]<uri> (optional) (Annotations attached to the entity.)
            "representation_of": string<uri> (optional) (Identifier for the item the spatial object represents.)
            "reference_organ": string<uri> (optional) (Identifier for the organ the spatial object represents.)
            
            "creator": (string) (optional)
            "creator_first_name": (string) (optional)
            "creator_last_name": (string) (optional)
            "creator_orcid": (string) (optional)
            "creation_date": (string)<date>,
            
            "x_dimension": (number) (required),
            "y_dimension": (number) (required),
            "z_dimension": (number) (required),
            "dimension_units": (string) (required) (default: millimeters),
            
            "label": (string) (optional)
            "comment": (string) (optional)
            "rui_rank": (integer) (optional)
            "sex": (string) (optional)
            "side": (string) (optional) (Used for organs that come in pairs to distinguish between them, i.e. kidneys, etc)
            "slice_count": (integer) (optional)
            "slice_thickness": (number) (optional)
            
            
            "placement": {
                "@id": (string) (required),
                "@type": (string) (required) (allowed value: SpatialEntity),
                "placement_date": string<date>,
                "x_translation": (number) (required),
                "y_translation": (number) (required),
                "z_translation": (number) (required),
                "translation_units": (number) (required) (allowed value: millimeters),
                "x_rotation": (number) (required),
                "y_rotation": (number) (required),
                "z_rotation": (number) (required),
                "rotation_units": (string) (required) (allowed value: degree),
                "x_scaling": (number) (required),
                "y_scaling": (number) (required),
                "z_scaling": (number)  (optional),
                "scaling_units": (string) (required) (allowed value: degree),
                "target": (string) (required),
        }
            
        }
    }
</pre>

In [28]:
get_spatial_placement_request = {
    "target_iri": "http://purl.org/ccf/latest/ccf.owl#VHFemale",
    "rui_location": {
        "@id": "http://purl.org/ccf/1.5/f7130161-ad0a-4801-b5a4-f6297e0f1869",
        "@type": "SpatialEntity",
        "creator": "Bruce Herr",
        "creator_first_name": "Bruce",
        "creator_last_name": "Herr",
        "creation_date": "2022-04-27",
        "ccf_annotations": ["http://purl.obolibrary.org/obo/UBERON_0002015", "http://purl.obolibrary.org/obo/UBERON_0000362", "http://purl.obolibrary.org/obo/UBERON_0004200", "http://purl.obolibrary.org/obo/UBERON_0001225", "http://purl.obolibrary.org/obo/UBERON_0001284", "http://purl.obolibrary.org/obo/UBERON_0002189"],
        "x_dimension": 11,
        "y_dimension": 12,
        "z_dimension": 13,
        "dimension_units": "millimeter",
        "placement": {
            "@id": "http://purl.org/ccf/1.5/f7130161-ad0a-4801-b5a4-f6297e0f1869_placement",
            "@type": "SpatialPlacement",
            "target": "http://purl.org/ccf/latest/ccf.owl#VHFLeftKidneyV1.1",
            "placement_date": "2022-04-27",
            "x_scaling": 1,
            "y_scaling": 1,
            "z_scaling": 1,
            "scaling_units": "ratio",
            "x_rotation": -61,
            "y_rotation": 50,
            "z_rotation": -84,
            "rotation_order": "XYZ",
            "rotation_units": "degree",
            "x_translation": 20.149,
            "y_translation": 109.963,
            "z_translation": 38.79,
            "translation_units": "millimeter"
        }
    }
}
try:
    # Get spatial placement
    api_response = api_instance.get_spatial_placement(get_spatial_placement_request)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

MaxRetryError: HTTPSConnectionPool(host='apps.humanatlas.io', port=443): Max retries exceeded with url: /api/v1/get-spatial-placement (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

***
<a id='rui_loc'></a>
#### Get all hubmap rui locations (if enabled)
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [ ]:
try:
    api_response = api_instance.gtex_rui_locations(cache = True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='ontology_term'></a>
#### Get number of ontology term occurrences for a search
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) 
</pre>

In [29]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.ontology_term_occurences(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

MaxRetryError: HTTPSConnectionPool(host='apps.humanatlas.io', port=443): Max retries exceeded with url: /api/v1/ontology-term-occurences?age=%7B%22min%22%3A%2045.0%2C%20%22max%22%3A%2060.0%7D&bmi=%7B%22min%22%3A%2065.0%2C%20%22max%22%3A%2083.0%7D&cache=true&ontology-terms=http://purl.obolibrary.org/obo/UBERON_0000955&cell-type-terms=http://purl.obolibrary.org/obo/CL_0000000&sex=female&spatial=%7B'x':%2084.0,%20'y':%20146.0,%20'z':%2053.0,%20'radius':%2012.0,%20'target':%20'http://purl.org/ccf/latest/ccf.owl (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

***
<a id='cell_term_occ'></a>
#### Get number of cell type term occurrences for a search
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) 
</pre>

In [ ]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.cell_type_term_occurences(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='ontology_nodes'></a>
#### Get ontology term tree nodes
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [ ]:
try:
    api_response = api_instance.ontology_tree_model(cache=True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='cell_nodes'></a>
#### Get cell type tree nodes
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [ ]:
try:
    api_response = api_instance.cell_type_tree_model(cache = False)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='providers'></a>
#### Get tissue provider names (for filtering)
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [ ]:
try:
    api_response = api_instance.provider_names(cache = True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='ref_organs'></a>
#### Get all reference organs
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [ ]:
try:
    api_response = api_instance.reference_organs(cache = True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='3d_scene'></a>
#### Get all nodes to form the 3D scene for an organ
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional)
    organ-iri - (string) (required)
</pre>

In [ ]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
organ_iri = "http://purl.obolibrary.org/obo/UBERON_0004539"
token = ""

try:
    # Get aggregate results / statistics
    api_response = api_instance.reference_organ_scene(organ_iri=organ_iri, age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token
)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='nodes_3d'></a>
#### Get all nodes to form the 3D scene of reference body, organs, and tissues
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional)
</pre>

In [ ]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    # Get aggregate results / statistics
    api_response = api_instance.scene(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token
)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='tech'></a>
#### Get technology names (for filtering)
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [ ]:
try:
    api_response = api_instance.technology_names(cache = False)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='tissue_block'></a>
#### Get Tissue Block Results
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional)
</pre>

In [ ]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.tissue_blocks(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token
)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='gtex_rui'></a>
#### Get all GTEx rui locations (if enabled)
<pre>
Accepted Parameters:
    cache - (boolean) (optional) 
</pre>

In [ ]:
try: 
    api_response = api_instance.gtex_rui_locations(cache = False)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

***
<a id='token_usage'></a>
#### Token Usage

In [ ]:
token = 'abbee123099245'
result = api_instance.db_status(token=token)
pprint(result)